In [1]:
from flask import Flask, request, abort

from linebot import (
    LineBotApi, WebhookHandler
)
from linebot.exceptions import (
    InvalidSignatureError
)
from linebot.models import (
    MessageEvent, TextMessage, TextSendMessage, ImageMessage,ImageSendMessage
)


app = Flask(__name__,static_url_path = "/images" , static_folder = "./images/" )


# create a instance for line
# line_bot_api 用來處理消息
line_bot_api = LineBotApi('YOUR_CHANNEL_ACCESS_TOKEN')

# 用來接收外部消息
handler = WebhookHandler('YOUR_CHANNEL_SECRET')

@app.route("/", methods=['POST'])
def callback():
    # get X-Line-Signature header value
    signature = request.headers['X-Line-Signature']

    # get request body as text
    body = request.get_data(as_text=True)
    app.logger.info("Request body: " + body)

    # handle webhook body 驗證封包是否來自Line
    try:
        handler.handle(body, signature)
    except InvalidSignatureError:
        abort(400)

    return 'OK'
    
@handler.add(MessageEvent, message=ImageMessage)
def handle_message(event):
    line_bot_api.reply_message(
        event.reply_token,
        TextSendMessage(text='Image has Upload'+ ' ' + event.message.id))
    message_content = line_bot_api.get_message_content(event.message.id)
    with open('./images/'+event.message.id+'.jpg', 'wb') as fd:
      for chunk in message_content.iter_content():
          fd.write(chunk)
    
if __name__ == "__main__":
    app.run(host='0.0.0.0')

 * Running on http://0.0.0.0:5000/ (Press CTRL+C to quit)
172.17.0.1 - - [01/May/2017 11:35:56] "POST / HTTP/1.1" 200 -
192.168.114.1 - - [01/May/2017 11:37:05] "GET /images/6019948694299 HTTP/1.1" 404 -
192.168.114.1 - - [01/May/2017 11:37:21] "GET /images/6019948694299.jpg HTTP/1.1" 200 -
